# producing database
# read files with no rows dropped, concat together 

In [2]:
import pandas as pd
import numpy as np
import geopandas
import matplotlib.pyplot as plt
from pandas.io.parsers import read_csv


In [3]:
sbg=pd.read_excel('half-way/complete/'+'SBG'+'.xlsx')
sbg=sbg.loc[:,['year','PE','tech_type', 'Bezirk','Gemeinde','KG_NR','KG']]
sbg['BL']='Salzburg'

In [4]:
karn=pd.read_excel('half-way/complete/'+'karn'+'.xlsx')
karn=karn.loc[:,['year','PE','tech_type', 'Bezirk','Gemeinde','KG_NR','KG']]
karn['BL']='Carintia'


In [5]:
noe=pd.read_excel('half-way/complete/'+'noe'+'.xlsx')
noe=noe[['INBETRIEBNAHME','EW60','tech_type','BEZIRK']]
noe.rename(columns={'INBETRIEBNAHME':'year','EW60':'PE','BEZIRK':'Bezirk' },inplace=True)
noe['BL']='Lower-Austria'
noe[noe.tech_type=='Unbekannt'].PE.sum()
noe.groupby('tech_type').PE.sum()

tech_type
3-k           2951.0
Andere        1958.0
BKF           2616.0
Bel.         20103.0
Fest           250.0
MBR             30.0
PKA          11932.0
RBC             78.0
SBR          53072.0
Tropf         1546.0
Unbekannt    38112.0
Wirbel          26.0
Name: PE, dtype: float64

In [6]:
tirol=pd.read_excel('half-way/complete/'+'tirol'+'.xlsx')
tirol=tirol[['year','PE','tech_type', 'Bezirk','Gemeinde','KG_NR','KG']]
tirol['BL']='Tirol'
tirol

,year,PE,tech_type,Bezirk,Gemeinde,KG_NR,KG,BL
0,2014,31.0,PKA,Imst,70217,80009,Pitztal,Tirol
1,2013,8.0,PKA,Imst,70210,80007,Mils bei Imst,Tirol
2,2008,26.5,PKA,Imst,70207,80006,Karrösten,Tirol
3,2007,5.0,PKA,Imst,70221,80111,Stams,Tirol
4,2002,5.0,PKA,Imst,70215,80106,Rietz,Tirol
...,...,...,...,...,...,...,...,...
1289,1958,0.0,3-k,Schwaz,70936,87011,Vomp,Tirol
1290,1956,0.0,3-k,Schwaz,70915,87110,Hart,Tirol
1291,1956,0.0,3-k,Schwaz,70926,87007,Schwaz,Tirol
1292,1955,10.0,3-k,Schwaz,70939,87014,Wiesing,Tirol


In [7]:
steyr=pd.read_excel('half-way/complete/'+'steyr'+'.xlsx')
steyr=steyr[['year','PE','tech_type', 'Gemeinde','KG_NR','KG']]
steyr['BL']='Stirya'
steyr

,year,PE,tech_type,Gemeinde,KG_NR,KG,BL
0,0,0,3-k,0,NaN,NaN,Stirya
1,0,0,3-k,0,NaN,NaN,Stirya
2,0,0,3-k,0,NaN,NaN,Stirya
3,0,0,0,0,NaN,NaN,Stirya
4,0,0,Unbekannt,0,NaN,NaN,Stirya
...,...,...,...,...,...,...,...
10713,2005,6,Bel.,62390 Straden,66213.0,Kronnersdorf,Stirya
10714,2014,5,Bel.,62390 Straden,62138.0,Muggendorf,Stirya
10715,2006,5,Bel.,62390 Straden,66243.0,Wieden,Stirya
10716,2007,0,Bel.,62390 Straden,66332.0,Radochen,Stirya


In [8]:
oebo=pd.read_excel('half-way/complete/'+'oebo'+'.xlsx')
oebo=oebo[['INBETRIEBNAHME','EW60','Verfahren', 'ANLAGENAME']]
oebo.rename(columns={'INBETRIEBNAHME':'year','EW60':'PE','Verfahren':'tech_type'},inplace=True)
oebo['BL']='Upper-Austria'
oebo.groupby('tech_type').PE.sum()

tech_type
Belebung      12593
Belebung +     1344
Filtration      180
PF             8799
SBR           15166
SBR +          5703
Sonstig        2237
mechanisch     2150
Name: PE, dtype: int64

In [9]:
#load minor states
BRG=pd.read_excel('half-way/complete/BRG.xlsx')
VRB=pd.read_excel('half-way/complete/VBG.xlsx')
VIE=pd.read_excel('half-way/complete/wien.xlsx')

In [10]:
files=[karn,steyr, noe,tirol,oebo,sbg, VIE,VRB,BRG]

In [11]:
data=pd.concat(files)

### remove mistkaes, unify categories


In [12]:
data.replace('Bewilligungzeitpunkt abgelaufen oder unbekannt',0,inplace=True)
data.replace('Bescheid mangelhaft',0,inplace=True)
data.replace('m³ pro Tag x 1000 / 150',0,inplace=True)
data.replace('[Liter pro Sekunde x 3600 (= 1 Std.) x 10] / 150',0,inplace=True)
data=data.fillna(0).astype({'PE':int})



In [13]:
data.tech_type=data.tech_type.replace('0', 'Unbekannt')
data.tech_type=data.tech_type.replace('Belebung', 'Bel.')
data.tech_type=data.tech_type.replace('Belebung +', 'Bel.')
data.tech_type=data.tech_type.replace('PF', 'PKA')
data.tech_type=data.tech_type.replace('Rotation', 'RBC')
data.tech_type=data.tech_type.replace('SBR +', 'SBR')
data.tech_type=data.tech_type.replace('Sonstig', 'Andere')
data.tech_type=data.tech_type.replace('mechanisch', '3-k')
data.tech_type=data.tech_type.replace('Filtersack', 'Andere')
data.tech_type=data.tech_type.replace('Kompost', 'Andere')


In [14]:
data.groupby('tech_type').PE.sum()

tech_type
0                 1550
3-k              78061
Andere            6875
BKF              15228
Bel.            113383
Fest              5070
Filterkammer        76
Filtration         180
MBR                308
PKA              60615
RBC               1354
SBR             101221
Tauch              367
Tropf             9589
Unbekannt        47894
Wirbel             416
Name: PE, dtype: int32

In [15]:
data[data.BL=='Lower-Austria'].groupby('tech_type').PE.sum()

tech_type
3-k           2951
Andere        1958
BKF           2616
Bel.         20103
Fest           250
MBR             30
PKA          11932
RBC             78
SBR          53072
Tropf         1546
Unbekannt    38112
Wirbel          26
Name: PE, dtype: int32

In [16]:
data=data.astype({'year':int})

# BEzirk, Gemeinde and KG_NR need to be cleaned and unified. either number or string

In [17]:
data.to_excel('final/database.xlsx',index=False)

In [18]:
import pandas as pd
test=pd.read_excel('final/database.xlsx')
test.groupby('BL').count()

,year,PE,tech_type,Bezirk,Gemeinde,KG_NR,KG,ANLAGENAME,Adresse,ID Nr.
BL,,,,,,,,,,
Burgenland,23,23,23,23,23,23,23,23,23,23
Carintia,7293,7293,7293,7293,7293,7293,7293,7293,7293,7293
Lower-Austria,5029,5029,5029,5029,5029,5029,5029,5029,5029,5029
Salzburg,1732,1732,1732,1732,1732,1732,1732,1732,1732,1732
Stirya,10718,10718,10718,10718,10718,10718,10718,10718,10718,10718
Tirol,1294,1294,1294,1294,1294,1294,1294,1294,1294,1294
Upper-Austria,2633,2633,2633,2633,2633,2633,2633,2633,2633,2633
Vienna,25,25,25,25,25,25,25,25,25,25
Vorarlberg,156,156,156,156,156,156,156,156,156,156


In [19]:
test.groupby('BL').count()

,year,PE,tech_type,Bezirk,Gemeinde,KG_NR,KG,ANLAGENAME,Adresse,ID Nr.
BL,,,,,,,,,,
Burgenland,23,23,23,23,23,23,23,23,23,23
Carintia,7293,7293,7293,7293,7293,7293,7293,7293,7293,7293
Lower-Austria,5029,5029,5029,5029,5029,5029,5029,5029,5029,5029
Salzburg,1732,1732,1732,1732,1732,1732,1732,1732,1732,1732
Stirya,10718,10718,10718,10718,10718,10718,10718,10718,10718,10718
Tirol,1294,1294,1294,1294,1294,1294,1294,1294,1294,1294
Upper-Austria,2633,2633,2633,2633,2633,2633,2633,2633,2633,2633
Vienna,25,25,25,25,25,25,25,25,25,25
Vorarlberg,156,156,156,156,156,156,156,156,156,156
